In [1]:
import tables
import numpy as np
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

In [2]:
hdf5_path = '/Users/chetan/Documents/Git Projects/Machine Learning/Computer Vision Problem/Dataset/dataset.hdf5'  # address to where you want to save the hdf5 file
subtract_mean = True
batch_size = 50
nb_class = 2

In [3]:
hdf5_file = tables.open_file(hdf5_path, mode='r')
# subtract the training mean
if subtract_mean:
    mm = hdf5_file.root.train_mean[0]
    mm = mm[np.newaxis, ...]

# Total number of samples
train_data = np.array(hdf5_file.root.train_img)
train_label = np.array(hdf5_file.root.train_labels)

test_data = np.array(hdf5_file.root.test_img)
test_label = np.array(hdf5_file.root.test_labels)

val_data = np.array(hdf5_file.root.val_img)
val_label = np.array(hdf5_file.root.val_labels)

print('train data:',train_data.shape,' train_label',train_label.shape)
print('test_data:',test_data.shape,' test_label:',test_label.shape)
print('val_data:',val_data.shape,' val_label:',val_label.shape)

# create list of batches to shuffle the data
#     batches_list = list(range(int(ceil(float(data_num) / batch_size))))
#     shuffle(batches_list)

#     # loop over batches
#     for n, i in enumerate(batches_list):
#         i_s = i * batch_size  # index of the first image in this batch
#         i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
#         print('i_s:',i_s,' i_e:',i_e)

#         # read batch images and remove training mean
#         images = hdf5_file.root.train_img[i_s:i_e]
#         print('len:',len(images))
#         if subtract_mean:
#             images -= mm

#         # read labels and convert to one hot encoding
#         labels = hdf5_file.root.train_labels[i_s:i_e]
#         labels_one_hot = np.zeros((len(images), nb_class))
#         labels_one_hot[np.arange(len(images)), labels] = 1

#         print(n+1, '/', len(batches_list))

#         print (labels[0], labels_one_hot[0, :])
#         plt.imshow(images[0])
#         plt.show()

#         if n == 5:  # break after 5 batches
#             break

train data: (259, 128, 128, 3)  train_label (259,)
test_data: (87, 128, 128, 3)  test_label: (87,)
val_data: (86, 128, 128, 3)  val_label: (86,)


In [4]:
from keras.utils import np_utils

# one-hot encode the labels
num_classes = len(np.unique(train_label))
train_label = np_utils.to_categorical(train_label, num_classes)
test_label = np_utils.to_categorical(test_label, num_classes)
val_label = np_utils.to_categorical(val_label, num_classes)

# print shape of training set
print('num_classes:', num_classes)

# print number of training, validation, and test images
print(train_label.shape, 'train samples')
print(test_label.shape, 'test samples')
print(val_label.shape, 'validation samples')

/Applications/Anaconda/anaconda3/envs/Python35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Applications/Anaconda/anaconda3/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


num_classes: 2
(259, 2) train samples
(87, 2) test samples
(86, 2) validation samples


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
__________

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
hist = model.fit(train_data, train_label, batch_size=None, epochs=20,
          validation_data=(val_data, val_label),callbacks=[checkpointer], 
          verbose=1, shuffle=True)

Coming here
Train on 259 samples, validate on 86 samples
Epoch 1/20
batch size here: [(0, 32), (32, 64), (64, 96), (96, 128), (128, 160), (160, 192), (192, 224), (224, 256), (256, 259)]
259/259 [==============================] - 6s 24ms/step - loss: 5.4186 - acc: 0.5946 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00001: val_loss improved from inf to 5.24775, saving model to model.weights.best.hdf5
Epoch 2/20
batch size here: [(0, 32), (32, 64), (64, 96), (96, 128), (128, 160), (160, 192), (192, 224), (224, 256), (256, 259)]
259/259 [==============================] - 4s 16ms/step - loss: 6.0365 - acc: 0.6255 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00002: val_loss did not improve
Epoch 3/20
batch size here: [(0, 32), (32, 64), (64, 96), (96, 128), (128, 160), (160, 192), (192, 224), (224, 256), (256, 259)]
259/259 [==============================] - 4s 17ms/step - loss: 6.0365 - acc: 0.6255 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00003: val_loss did not improve
Epoch 4/20
batch si

In [8]:
model.load_weights('model.weights.best.hdf5')

In [9]:
score = model.evaluate(test_data, test_label, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.724137934460037
